In [2]:
import re
from elasticsearch import Elasticsearch, helpers
import numpy as np

from config import CLOUD_ID, ADMIN, PASSWORD


es = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=(ADMIN, PASSWORD)  # Replace with your Cloud credentials
)

index_name = "doc_zeta"
scroll_duration = "2m"
batch_size = 100

response = es.search(
    index=index_name,
    scroll=scroll_duration,
    size=batch_size,
    body={
        "_source": ["document_text", "document_url"],
        "query": {"match_all": {}}
    }
)

scroll_id = response['_scroll_id']
hits = response['hits']['hits']

# Store results as a list of tuples
result_list = []

while hits:
    for doc in hits:
        source = doc['_source']
        text = source.get('document_text', '')
        url = source.get('document_url', 'N/A')

        word_count = len(re.findall(r'\S+', text))
        char_count = len(text)

        result_list.append((url, word_count, char_count))

    # Fetch next batch
    response = es.scroll(scroll_id=scroll_id, scroll=scroll_duration)
    scroll_id = response['_scroll_id']
    hits = response['hits']['hits']

es.clear_scroll(scroll_id=scroll_id)

# Convert to NumPy array
np_array = np.array(result_list, dtype=object)


/tmp/ipykernel_5359/264625299.py:17: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific paremeters.
  response = es.search(


In [5]:
import pandas as pd

df = pd.DataFrame(np_array, columns=['document_url', 'word_count', 'char_count'])

df.describe()

,document_url,word_count,char_count
count,45562,45562,45562
unique,45377,10694,28304
top,N/A,0,0
freq,151,196,162


In [9]:
df['word_count'].describe(include='all')


count     45562
unique    10694
top           0
freq        196
Name: word_count, dtype: int64

In [ ]:
df['char_count'].describe()




In [11]:
mode_words = df['word_count'].mean()

# Max
max_words = df['word_count'].max()

# Min
min_words = df['word_count'].min()

print("Mode:", mode_words)
print("Max:", max_words)
print("Min:", min_words)

Mode: 3645.8073175014265
Max: 49665
Min: 0


In [26]:

df[(df['word_count'] < 10000) & (df['word_count'] > 9000)]


,document_url,word_count,char_count
71,shiromani-gurudwaraprabandhak-committee-amrits...,9744,55521
332,colgate-palmolive-india-ltd-v-hindustan-lever-ltd,9492,56194
398,trust-of-india-and-anr-v-union-of-india-anr,9340,55130
450,ms-v-l-s-finance-ltd-v-s-p-gupta-and-anr,9348,56640
480,devji-vallabhbhai-tandel-etc-v-the-administrat...,9113,54267
...,...,...,...
44967,the-state-of-chattisgarh-anr-v-aman-kumar-singh-o,9601,57946
44997,the-state-of-west-bengal-v-debabrata-tiwari-or...,9811,62350
45029,yash-developers-v-harihar-krupa-co-operative-h...,9673,59977
45277,r-kandasamysince-dead-v-trksarawathy,9669,64092


In [24]:
df.sort_values(by='word_count', ascending=False).head(10)

# df.sort_values(by='word_count', ascending=False).head(10)['document_url']


,document_url,word_count,char_count
14605,kc-vasanth-kumar-another-v-state-of-karnataka,49665,304429
38285,union-of-india-and-another-v-tulsiram-patel-an...,49544,290870
26944,ram-narain-popli-v-central-bureau-of-investiga...,49472,290608
11244,the-bengal-immunity-company-limited-v-the-stat...,49384,286126
4431,the-ahmedabad-st-xaviers-college-society-anr-e...,49370,306009
17019,smt-ujjam-bai-v-state-of-uttar-pradesh,49354,287501
1216,llfe-insurance-corporation-of-lndla-v-escorts-ltd,49271,294170
36453,express-newspapers-private-ltdand-another-v-the-u,49159,291835
15097,itc-ltd-etc-v-state-of-karnataka-ors,49132,285190
7548,additional-district-magistrate-jabalpur-v-s-s-...,49131,287124
